In [33]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px


In [34]:
spacex_df = pd.read_csv("C://spacex_launch_dash.csv")
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [35]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [37]:
JupyterDash.infer_jupyter_proxy_config()

In [39]:
 # needs to be run again in a separate cell due to a jupyterdash bug
JupyterDash.infer_jupyter_proxy_config()

In [46]:
# Create a dash application
app = JupyterDash(__name__)

app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                html.Div(dcc.Dropdown(
                                    id='site-dropdown',
                                    options = [{'label': 'ALL', 'value': 'ALL'}] + [{'label': idx, 'value': idx} for idx in spacex_df['Launch Site'].unique()],
                                    placeholder="Select a Launch Site here",
                                    searchable=True,
                                    style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'textAlignLast': 'center'},
                                    ),),
                                html.Br(),
                                html.Br(),
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Div([
                                    dcc.RangeSlider(
                                        id='payload-slider',
                                        min=0,
                                        max=10000,
                                        step=1000,
                                        value=[min_payload, max_payload],
                                        marks = {0: {'label': '0kg'},
                                                 1000: {'label': '1000kg'},
                                                 2000: {'label': '2000kg'},
                                                 3000: {'label': '3000kg'},
                                                 4000: {'label': '4000kg'},
                                                 5000: {'label': '5000kg'},
                                                 6000: {'label': '6000kg'},
                                                 7000: {'label': '7000kg'},
                                                 8000: {'label': '8000kg'},
                                                 9000: {'label': '9000kg'},
                                                 10000: {'label': '10000kg'}},
                                        ),
                                    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                    ])
                                ])

# add callback decorator
@app.callback([Output(component_id='success-pie-chart', component_property='figure'),
               Output(component_id='success-payload-scatter-chart', component_property='figure')],
              [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])#

# Add computation to callback function and return graphs
def get_graph(entered_site, slider_value):
    if entered_site == 'ALL' or entered_site is None:
        dff = spacex_df
        chart_data = dff.groupby('Launch Site')['class'].count().reset_index()
        pie = go.Pie(labels = list(chart_data['Launch Site']), values = list(chart_data['class']))
        fig = go.Figure(data=pie)
        fig.update_layout(title='Success for All sites')
        scatter_title='Correlation between Payload and Success for all Sites'
    elif entered_site is not None:
        dff = spacex_df[spacex_df['Launch Site']==entered_site]
        successes = dff[dff['class']==1].shape[0]
        failures = dff[dff['class']==0].shape[0]
        pie = go.Pie(labels = ['Success', 'Failure'], values = [successes, failures])
        fig = go.Figure(data=pie)
        fig.update_layout(title="Success for '%s'" % entered_site)
        scatter_title="Correlation between Payload and Success for '%s'" % entered_site
    if slider_value is None:
        scatter_title = scatter_title + " with Payload >= 0kg"
        slider_value = 0.0
    else:
        scatter_title = scatter_title + " with Payload >= %dkg" % int(slider_value[0])
    scatter_data = dff[dff['Payload Mass (kg)']>=slider_value[0]]
    scat_fig = px.scatter(scatter_data, x='Payload Mass (kg)', y='class', title=scatter_title, color='Booster Version Category')
    return [fig, scat_fig]

# Run the app
if __name__ == '__main__':
    app.run_server(mode="inline", host="localhost", port=8058, debug=True)

Double-click **here** for the solution.

<!-- The answer is below:
    
# Create a dash application
app = JupyterDash(__name__)

# Read the airline data into pandas dataframe
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})

# TODO
# Get the layout of the application and adjust it.
# Create an outer division using html.Div and add title to the dashboard using html.H1 component
# Add a html.Div and core input text component
# Finally, add graph component.
app.layout = html.Div(children=[ html.H1('Airline Performance Dashboard', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 40}),
                                html.Div(["Input Year: ", dcc.Input(id='input-year', value='2010', 
                                type='number', style={'height':'50px', 'font-size': 35}),], 
                                style={'font-size': 40}),
                                html.Br(),
                                html.Br(),
                                html.Div(dcc.Graph(id='line-plot')),
                                ])

# add callback decorator
@app.callback( Output(component_id='line-plot', component_property='figure'),
               Input(component_id='input-year', component_property='value'))

# Add computation to callback function and return graph
def get_graph(entered_year):
    # Select 2019 data
    df =  airline_data[airline_data['Year']==int(entered_year)]
    
    # Group the data by Month and compute average over arrival delay time.
    line_data = df.groupby('Month')['ArrDelay'].mean().reset_index()

    fig = go.Figure(data=go.Scatter(x=line_data['Month'], y=line_data['ArrDelay'], mode='lines', marker=dict(color='green')))
    fig.update_layout(title='Month vs Average Flight Delay Time', xaxis_title='Month', yaxis_title='ArrDelay')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(mode="inline", host="localhost")
    
-->


## Summary

Congratulations for completing your dash interactivity lab. 

In this lab, you have learnt how to make dash HTML and core components interact with each other for creating dashboard.


## Author

[Saishruthi Swaminathan](https://www.linkedin.com/in/saishruthi-swaminathan?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork-20297740&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ) 


## Changelog

| Date       | Version | Changed by | Change Description                   |
| ---------- | ------- | ---------- | ------------------------------------ |
| 12-18-2020 | 1.0     | Nayef      | Added dataset link and upload to Git |


## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
